In [5]:
#Loading in Pretrained Model
import sys
import os
sys.version
import keras
from keras import callbacks
from keras import backend as K
from keras.applications.resnet50 import ResNet50

resnet_base = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


# Loading the VGG19 Pre-Trained Model

In [6]:
resnet_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 75, 75, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

# Model V3

## Add Second Hidden Layer and Re Train the last 2 layers of Resnet

### Same architecture as VGG16 & VGG19 models

In [7]:
#Model with Conv Base Included
from keras import models
from keras import layers
from keras import optimizers
model_rn = models.Sequential()
model_rn.add(resnet_base)
model_rn.add(layers.Flatten())
model_rn.add(layers.Dense(256, activation='relu', input_dim=5 * 5 * 2048))
model_rn.add(layers.Dropout(0.5))
model_rn.add(layers.Dense(128, activation='relu'))
model_rn.add(layers.Dropout(0.5))
model_rn.add(layers.Dense(10, activation = 'softmax'))

class MyCallback(callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        print(K.eval(self.model.optimizer.lr))
PrintLR=MyCallback()

savebest=callbacks.ModelCheckpoint(filepath='/home/jupyter/Saved_Models/checkpoint-{val_acc:.2f}.h5',monitor='val_acc',save_best_only=True)
ReduceLR=callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.75, verbose=0, mode='auto', cooldown=20, min_lr=1e-7)
Earlystop = callbacks.EarlyStopping(monitor='val_loss', patience=20)

callbacks_list=[savebest,ReduceLR,PrintLR]

model_rn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               13107456  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total para

In [17]:
# Set last 2 layers of Resnet as trainable

resnet_base.trainable = True

set_trainable = False
for layer in resnet_base.layers:
    if layer.name == 'res5b_branch2a' :
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
print(len(resnet_base.trainable_weights))

24


In [18]:
from keras.utils import multi_gpu_model
parallel_model = multi_gpu_model(model_rn, gpus=2)
parallel_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

ResourceExhaustedError: OOM when allocating tensor with shape[128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dense_2/bias/Assign (defined at /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:402)  = Assign[T=DT_FLOAT, _grappler_relax_allocator_constraints=true, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_2/bias, dense_2/Const)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_2/bias/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/p

## Splitting Data into Train and Validate Sets (80% Train % 20% Validation)

In [19]:
from keras.preprocessing.image import ImageDataGenerator


base_dir = '/home/jupyter/Data'
train_dir = os.path.join(base_dir, 'train')

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
    validation_split=0.2
    )



train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        subset="training",
        shuffle = True
        )

validation_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        subset="validation",
        shuffle = True
        )

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


# Run Model V3

In [ ]:
history_v2 = parallel_model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=500,
      validation_data=validation_generator,
      validation_steps=50,
      callbacks=callbacks_list,
      #use_multiprocessing = True,
      #workers = 4,
      verbose=2)

print("Finished")

Epoch 1/500
 - 70s - loss: 2.3058 - acc: 0.0985 - val_loss: 2.3025 - val_acc: 0.1030
1e-05
Epoch 2/500


# Plot Training and Validation Accuracy of 50 Epoch model 

In [ ]:
import matplotlib.pyplot as plt
acc = history_v2.history['acc']
val_acc = history_v2.history['val_acc']
loss = history_v2.history['loss']
val_loss = history_v2.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'g', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Save Model

In [ ]:
import model_util as util
model_json = model_v3.to_json()
with open("/home/jupyter/Saved_Models/Resnet_500.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_v3.save_weights("/home/jupyter/Saved_Models/Resnet_500.h5")
print("Saved model to disk")

In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator
#test data
base_dir = '/home/jupyter/Data'
test_dir = os.path.join(base_dir, 'test')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        '/home/jupyter/Data/TEST',
        target_size=(150, 150),
        batch_size=20,
        classes=['test'],  # only data, no labels
        shuffle=False
       )


In [ ]:
from model_util import create_submission

In [ ]:
create_submission(model_v3,test_generator,'/home/jupyter/Submission/','sample_submissionresnet.csv')

In [ ]:
import time
start = time.time()
probabilities = loaded_model.predict_generator(test_generator, len(test_generator),verbose=1)
end = time.time()
print(end - start)

In [ ]:
output_dir = '/home/jupyter/Submission/'
import pandas as pd
df_prob = pd.DataFrame(probabilities)
df_label = pd.DataFrame(test_generator.filenames[0:79727])
df = pd.concat([df_label,df_prob],axis=1)
df.columns = ['img','c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
df.head()
df.to_csv(output_dir + 'sample_submission6.csv',index=False)